In [1]:
import h2o
import numpy as np
import pandas as pd
import impyute as impy

from sklearn.preprocessing import LabelBinarizer, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer

from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators import H2OXGBoostEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.grid.grid_search import H2OGridSearch

###### Initialize H2O

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 17.0.6+9-LTS-190, mixed mode, sharing)
  Starting server from D:\programfiles\anaconda\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\amr_a\AppData\Local\Temp\tmp19bdvsvw
  JVM stdout: C:\Users\amr_a\AppData\Local\Temp\tmp19bdvsvw\h2o_amr_a_started_from_python.out
  JVM stderr: C:\Users\amr_a\AppData\Local\Temp\tmp19bdvsvw\h2o_amr_a_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_amr_a_ybhotp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.957 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


###### Read Data

In [9]:
# use low_memory=False to avoid warning which happens when pandas tries to guess data types for each column
final_df = pd.read_csv("../Data/Processed.csv", low_memory=False)
final_df.head()

,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Loan_Status
0,12232.0,0.0,728.0,0.0,0.0,46643.0,0.0,777.39,18.0,10.000000,12.0,0.0,82.231381,7946.0,0.0,0.0,Loan Refused
1,25014.0,1.0,733.0,1.0,1.0,81099.0,0.0,892.09,26.7,45.346829,14.0,0.0,188.960313,77961.0,0.0,0.0,Loan Refused
2,16117.0,0.0,724.0,2.0,1.0,60438.0,1.0,1244.02,16.7,32.000000,11.0,1.0,106.183803,14815.0,1.0,0.0,Loan Refused
3,11716.0,0.0,740.0,3.0,0.0,34171.0,0.0,990.94,10.0,21.443697,21.0,0.0,83.719771,43533.0,0.0,0.0,Loan Refused
4,9789.0,1.0,686.0,1.0,1.0,47003.0,1.0,503.71,16.7,25.000000,13.0,1.0,130.049990,19553.0,1.0,0.0,Loan Refused


In [10]:
final_df.columns[:-1]

Index(['Current Loan Amount', 'Term', 'Credit Score', 'Years in current job',
       'Home Ownership', 'Annual Income', 'Purpose', 'Monthly Debt',
       'Years of Credit History', 'Months since last delinquent',
       'Number of Open Accounts', 'Number of Credit Problems',
       'Current Credit Balance', 'Maximum Open Credit', 'Bankruptcies',
       'Tax Liens'],
      dtype='object')

In [11]:
#y = LabelBinarizer().fit_transform(final_df['Loan Status']).ravel()
column_names_to_standardize = list(final_df.columns[:-1])
data_scaled = StandardScaler().fit_transform(final_df[column_names_to_standardize].values)

data_temp = pd.DataFrame(data_scaled, columns=column_names_to_standardize, index = final_df.index)
final_df[column_names_to_standardize] = data_temp

In [12]:
final_df.head()

,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Loan_Status
0,-0.186918,-0.57303,0.068863,-1.114764,-1.056209,-0.314373,-0.364609,-0.295338,-0.044731,-1.091910,0.181035,-0.338742,-0.579308,-0.798455,-0.322950,-0.109584,Loan Refused
1,1.483498,1.74511,0.266450,-0.804658,0.519685,0.532731,-0.364609,-0.111707,1.189463,0.835650,0.585570,-0.338742,1.558342,1.838258,-0.322950,-0.109584,Loan Refused
2,0.320794,-0.57303,-0.089206,-0.494551,0.519685,0.024778,0.215558,0.451720,-0.229151,0.107811,-0.021232,1.772105,-0.099570,-0.539774,2.393046,-0.109584,Loan Refused
3,-0.254351,-0.57303,0.543071,-0.184445,-1.056209,-0.620998,-0.364609,0.046548,-1.179622,-0.467853,2.001441,-0.338742,-0.549497,0.541725,-0.322950,-0.109584,Loan Refused
4,-0.506181,1.74511,-1.590865,-0.804658,0.519685,-0.305522,0.215558,-0.733489,-0.229151,-0.273918,0.383302,1.772105,0.378440,-0.361344,2.393046,-0.109584,Loan Refused


In [15]:
y = LabelBinarizer().fit_transform(final_df['Loan_Status']).ravel()
y

array([1, 1, 1, ..., 0, 1, 0])

###### Model Training

In [16]:
hf = h2o.H2OFrame(final_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [29]:
predictors = list(final_df.columns[:-1])
response = "Loan_Status"

In [30]:
gbm_params1 = {'learn_rate': [0.01, 0.1],
                'max_depth': [5, 9],
                'ntrees': [300, 500],
                'sample_rate': [0.8, 1.0],
                'col_sample_rate': [0.5, 1.0]}

In [31]:
gbm_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator,
                          grid_id='gbm_grid1',
                          hyper_params=gbm_params1)

In [32]:
train, test = hf.split_frame(ratios = [.8], seed = 1234)

In [33]:
gbm_grid1.train(x=predictors, y=response, training_frame=train, seed=1234)

gbm Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


,col_sample_rate,learn_rate,max_depth,ntrees,sample_rate,model_ids,logloss
,1.0,0.1,9.0,500.0,0.8,gbm_grid1_model_16,0.3735738
,1.0,0.1,9.0,300.0,0.8,gbm_grid1_model_8,0.3735738
,0.5,0.1,9.0,500.0,0.8,gbm_grid1_model_15,0.3774627
,0.5,0.1,9.0,300.0,0.8,gbm_grid1_model_7,0.3774627
,1.0,0.1,9.0,300.0,1.0,gbm_grid1_model_24,0.3846239
,1.0,0.1,9.0,500.0,1.0,gbm_grid1_model_32,0.3846239
,0.5,0.1,9.0,300.0,1.0,gbm_grid1_model_23,0.3913150
,0.5,0.1,9.0,500.0,1.0,gbm_grid1_model_31,0.3913150
,1.0,0.01,9.0,500.0,1.0,gbm_grid1_model_30,0.4284238
,1.0,0.01,9.0,500.0,0.8,gbm_grid1_model_14,0.4291626


In [34]:
gbm_gridperf1 = gbm_grid1.get_grid(sort_by='aucpr', decreasing=True)
gbm_gridperf1

,col_sample_rate,learn_rate,max_depth,ntrees,sample_rate,model_ids,aucpr
,1.0,0.1,9.0,500.0,0.8,gbm_grid1_model_16,0.8328929
,1.0,0.1,9.0,300.0,0.8,gbm_grid1_model_8,0.8328929
,0.5,0.1,9.0,500.0,0.8,gbm_grid1_model_15,0.8259539
,0.5,0.1,9.0,300.0,0.8,gbm_grid1_model_7,0.8259539
,1.0,0.1,9.0,300.0,1.0,gbm_grid1_model_24,0.8153761
,1.0,0.1,9.0,500.0,1.0,gbm_grid1_model_32,0.8153761
,0.5,0.1,9.0,300.0,1.0,gbm_grid1_model_23,0.8024522
,0.5,0.1,9.0,500.0,1.0,gbm_grid1_model_31,0.8024522
,1.0,0.01,9.0,500.0,1.0,gbm_grid1_model_30,0.7349202
,1.0,0.01,9.0,500.0,0.8,gbm_grid1_model_14,0.7344326


###### Pick the best model from gridsearch

In [36]:
best_gbm1 = gbm_gridperf1.models[0]
perf = best_gbm1.model_performance(test)
perf

ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.17315595290802457
RMSE: 0.4161201183649074
LogLoss: 0.5080153884443578
Mean Per-Class Error: 0.32306216143058875
AUC: 0.7480657090263217
AUCPR: 0.5093603188859075
Gini: 0.4961314180526435

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2642982261474991
              Loan Given    Loan Refused    Error    Rate
------------  ------------  --------------  -------  ----------------
Loan Given    7765          4973            0.3904   (4973.0/12738.0)
Loan Refused  1297          3775            0.2557   (1297.0/5072.0)
Total         9062          8748            0.352    (6270.0/17810.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.264298     0.54631   253
max f2                       0.0992482    0.718242  349
max f0point5                 0.426861     0.509285  162
max accuracy                 0.519137     0.737282  116
max precision                0.926069     1         0
max recall                   0.0178115    1         386
max specificity              0.926069     1         0
max absolute_mcc             0.264298     0.319466  253
max min_per_class_accuracy   0.299528     0.673107  233
max mean_per_class_accuracy  0.264298     0.676938  253
max tns                      0.926069     12738     0
max fns                      0.926069     5070      0
max fps                      0.00248132   12738     399
max tps                      0.0178115    5072      386
max tnr                      0.926069     1         0
max fnr                      0.926069     0.999606  0
max fpr                      0.00248132   1         399
max tpr                      0.0178115    1         386

Gains/Lift Table: Avg response rate: 28.48 %, avg score: 28.06 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100505                   0.778691           2.47174   2.47174            0.703911         0.834023    0.703911                    0.834023            0.0248423       0.0248423                  147.174   147.174            0.0206815
2        0.0200449                   0.727613           2.34753   2.40981            0.668539         0.75212     0.686275                    0.793186            0.0234621       0.0483044                  134.753   140.981            0.0395118
3        0.0300393                   0.692643           2.18972   2.33658            0.623596         0.7097      0.665421                    0.765409            0.0218849       0.0701893                  118.972   133.658            0.0561368
4        0.0400337                   0.665193           2.2489    2.31469            0.640449         0.677181    0.659187                    0.743383            0.0224763       0.0926656                  124.89    131.469            0.0735888
5        0.0500281                   0.643182           1.69654   2.1912             0.483146         0.6536      0.624018                    0.725447            0.0169558       0.109621                   69.6536   119.12             0.0833222
6        0.1                         0.559463           2.06346   2.12737            0.58764          0.599507    0.605839                    0.662512            0.103115        0.212737                   106.346   112.737            0.157626
7        0.150028                    0.501268           1.76951   2.00804            0.503928         0.529806    0.571856     

In [37]:
gbm = H2OGradientBoostingEstimator(col_sample_rate=1.0, learn_rate=0.1, max_depth=9, ntrees=500, seed=1234)
gbm.train(x=predictors, y=response, training_frame=train)

perf = gbm.model_performance(test)
print(perf)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.17287746944270424
RMSE: 0.4157853646326482
LogLoss: 0.506915944273993
Mean Per-Class Error: 0.3196139664819688
AUC: 0.7486770501636228
AUCPR: 0.5083186770152344
Gini: 0.4973541003272457

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.26277828119012747
              Loan Given    Loan Refused    Error    Rate
------------  ------------  --------------  -------  ----------------
Loan Given    7667          5071            0.3981   (5071.0/12738.0)
Loan Refused  1223          3849            0.2411   (1223.0/5072.0)
Total         8890          8920            0.3534   (6294.0/17810.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.262778     0.550172  252
max f2    

In [38]:
model_path = h2o.save_model(
  gbm,
  path = "../models/",
  force = True,
  export_cross_validation_predictions = False
)
model_path

'C:\\Users\\amr_a\\Loan Eligibility\\Models\\GBM_model_python_1681068610365_2302'

In [40]:
h2o.load_model("../models/GBM_model_python_1681068610365_2302")

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_model_python_1681068610365_2302


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    117                117                         321019                 0            9            8.92308       1             396           213.88

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.12176688898026686
RMSE: 0.3489511269221907
LogLoss: 0.3846238805507943
Mean Per-Class Error: 0.18928833448419935
AUC: 0.9054470169667572
AUCPR: 0.8153760888828326
Gini: 0.8108940339335144

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.36224193431300533
              Loan Given    Loan Refused    Error    Rate
------------  ------------  --------------  -------  -----------------
Loan Given    43718         7281            0.1428   (7281.0/50999.0)
Loan Refused  4740          15361           0.2358   (4740.0/20101.0)
Total         48458         22642           0.1691   (12021.0/71100.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.362242     0.718761  202
max f2                       0.248995     0.799386  263
max f0point5                 0.486156     0.756129  140
max accuracy                 0.4341       0.845007  165
max precision                0.962112     1         0
max recall                   0.0534846    1         372
max specificity              0.962112     1         0
max absolute_mcc             0.400809     0.605008  182
max min_per_class_accuracy   0.331056     0.813044  218
max mean_per_class_accuracy  0.313165     0.814337  228
max tns                      0.962112     50999     0
max fns                      0.962112     20099     0
max fps                      0.00283751   50999     399
max tps                      0.0534846    20101     372
max tnr                      0.962112     1         0
max fnr                      0.962112     0.999901  0
max fpr                      0.00283751   1         399
max tpr                      0.0534846    1         372

Gains/Lift Table: Avg response rate: 28.27 %, avg score: 28.28 %
group    cumulative_data_fraction    lower_threshold    lift        cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ----------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.01                        0.810676           3.53714     3.53714            1                0.854878    1                           0.854878            0.0353714       0.0353714                  253.714   253.714            0.0353714
2        0.02                        0.763309           3.52719     3.53216            0.997187         0.785517    0.998594                    0.820198            0.0352719       0.0706433                  252.719   253.216            0.070604
3        0.03                        0.727957           3.53216     3.53216            0.998594         0.744137    0.998594                    0.794844            0.0353216       0.105965                   253.216   253.216            0.105906
4        0.04                        0.69991            3.45754     3.51351            0.977496         0.713298    0.993319                    0.774458            0.0345754       0.14054                    245.754   251.351           